<a href="https://colab.research.google.com/github/mirovoy465/colab/blob/main/ods.ai%20linear%20course%20homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Open ML Course: Линейные модели 2023**

# Домашнее задание 1

# Вопрос 1.
Создайте данные используя make_regression(n_samples=100, n_features=20, noise=10, random_state=42)
Что можно сказать о полученных данных?

In [22]:
from sklearn.datasets import make_regression

X, y =  make_regression(n_samples=100, n_features=20, noise=10)

Среднее признаков по модулю меньше единицы и стандартное отклонение около единицы (отличается не более, чем на 20%)

In [23]:
X.mean(), X.std()

(-0.0058133918653301255, 0.965571259146347)

Есть корреляции больше .5 между разными признаками

In [30]:
import numpy as np

correlation_matrix = np.corrcoef(X, rowvar=False)
correlation_matrix -= (np.isclose(correlation_matrix, 1)).astype(int)
(correlation_matrix > .5).any()

False

Для вычисления значений признаков требуется регуляризованная регрессия

In [58]:
# ?????? зачем вычислять признаки

Для лучшего предсказания признаки нужно отнормировать


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
scores = np.array([])

for iter in range(1000):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3)

  scaler = StandardScaler()
  X_train_sc = scaler.fit_transform(X_train)
  X_test_sc = scaler.fit_transform(X_test)

  linreg = LinearRegression()
  fit = linreg.fit(X_train, y_train)
  fit_sc = linreg.fit(X_train_sc, y_train)

  iter_scores = r2_score(fit.predict(X_test), y_test),\
                r2_score(fit.predict(X_test_sc), y_test)

  scores = np.append(scores, iter_scores, axis = 0)

scores.reshape(-1, 2).mean(axis=0)

array([0.98239305, 0.91953286])

без нормализации действительно получается лучше

# Вопрос 2.

Используя sklearn.linear_model постройте LinearRegression, Ridge(random_state=1), Lasso на дефолтных параметрах и ненормированных признаках. Для оценки регрессии используем R2 из функции score модели.